<a href="https://colab.research.google.com/github/JHyunjun/SNU/blob/main/KoGPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Install Packages

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.2 MB/s 
     |████████████████████████████████| 101 kB 10.4 MB/s 
     |████████████████████████████████| 596 kB 53.7 MB/s 
     |████████████████████████████████| 6.6 MB 48.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# 1. Import Packages

 - 본 실습에 필요한 패키지들을 불러옵니다.

In [3]:
from transformers import GPT2Model
from transformers import GPT2LMHeadModel
from transformers import PreTrainedTokenizerFast
import torch
from torch.utils.data import Dataset, DataLoader
import urllib
import pandas as pd

# 2. KoGPT2 Tokenizer

 - 사전 학습된 KoGPT2 Tokenizer를 불러옵니다.

In [4]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>', padding_side='right') 
sample_text = "근육이 커지기 위해서는"

tokens = tokenizer.tokenize(sample_text)
token_ids = tokenizer.encode(sample_text)

print(f' Sentence: {sample_text}')
print(f'   Tokens: {tokens}')
print(f'Token IDs: {token_ids}')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


 Sentence: 근육이 커지기 위해서는
   Tokens: ['▁근육이', '▁커', '지기', '▁위해서는']
Token IDs: [33245, 10114, 12748, 11357]


# 3. KoGPT2 Models

 - GPT2Model과 GPT2LMHeadModel을 불러옵니다.

## 3-1. GPT2Model

 - GPT2Model은 hidden state를 출력합니다.
 
 - 본 예제에서는 네 개의 토큰에 대한 768차원의 벡터가 도출됩니다.

In [5]:
gpt2_model = GPT2Model.from_pretrained('skt/kogpt2-base-v2')
hidden_states = gpt2_model(torch.tensor([token_ids]))
last_hidden_state = hidden_states[0]
print(last_hidden_state.shape) # 1 : Sentence갯수, 4 : Token의 갯수(근육이,커,지기,위해서는), 768 : 768차원의 벡터로 표현

Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([1, 4, 768])


## 3-2. GPT2LMHeadModel

 - GPT2LMHead는 next word prediction을 출력합니다.
 
 - 본 예제에서는 네 개의 토큰에 대한 51200 차원의 단어 확률 분포가 도출됩니다.

In [6]:
gpt2lm_model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
outputs = gpt2lm_model(torch.tensor([token_ids]))
next_word_predictions = outputs[0]
print(next_word_predictions.shape) #51200 : Vocab size

torch.Size([1, 4, 51200])


 - 단어 확률 분포에 대해 argmax를 취해 가장 높은 확률을 보이는 단어를 찾습니다.
 
 - 본 예제에서는 "무엇보다" 라는 단어가 가장 높은 확률을 나타냅니다.

In [7]:
next_word_distribution = next_word_predictions[0, -1, :]
next_word_id = torch.argmax(next_word_distribution) # Predict한값중 가장 Max값을 뽑아서 Return (INDEX)
next_word = tokenizer.decode(next_word_id) # 다음에 도출될 단어는 Above INDEX에 해당하는 단어다

print(f'Next word: {next_word}') # 근육이 커지기 위해서는 -> 무엇보다00

Next word: 무엇보다


# 4. Text Generation Examples (Pre-trained model)

 - 두 가지 Text Generation 방법을 실험해봅니다.

## 4-1. Greedy Search

 - Greedy Search는 가장 높은 확률의 단어를 Greedy하게 찾는 방식으로 텍스트를 생성합니다.

In [8]:
gen_ids = gpt2lm_model.generate(torch.tensor([token_ids]),
                           max_length=127,
                           repetition_penalty=2.0,
                           )

generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.
또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.
아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.
운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.
운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.



## 4-2. Beam Search

 - Beam Search는 매 step마다 num_beams 개 만큼의 Top word selection path를 찾습니다.


In [9]:
gen_ids = gpt2lm_model.generate(torch.tensor([token_ids]),
                           max_length=127,
                           repetition_penalty=2.0,
                           num_beams=5, 
                           )

generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

근육이 커지기 위해서는 피부 속 콜라겐과 엘라스틴의 생성을 촉진시키는 것이 중요하다.
콜라겐은 피부의 탄력을 유지하는 데 중요한 역할을 한다.
이러한 콜라겐의 생성을 촉진시키기 위해서는 피부에 충분한 수분을 공급해줘야 한다.
또한 피부를 촉촉하게 유지시켜주는 보습제를 꾸준히 섭취하는 것도 도움이 된다.
피부에 영양을 공급해주는 보습제로는 에센셜 오일이 있다.
에센셜 오일은 비타민 A, C, E가 풍부하게 함유돼 있어 노화방지에 도움을 주는 것으로 알려져 있다.
특히 에센셜 오일은 항산화 작용을 하는 활성산소를 억제해 피부 노화를 방지하는데 도움을 준다.



# 5. Fine tunning (Naver Movie review)

 - 네이버 영화 리뷰데이터를 활용하여 모델을 Fine Tuning 합니다.

## 5-1. Get Datasets

 - github으로부터 네이버 영화 리뷰데이터를 요청하여 내 pc에 저장합니다.
 
 - 데이터의 크기가 너무 큰 관계로, 본 실험에서는 테스트 데이터 셋만을 활용하여 모델을 학습시킵니다. 

In [10]:
def get_naver_review_examples():
    #urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
    urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

    #train_data = pd.read_table('ratings_train.txt')
    test_data = pd.read_table('ratings_test.txt')
    
    return test_data

In [11]:
naver_data = get_naver_review_examples()

In [12]:
naver_data

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


 - Dataset Loader를 정의합니다.

In [13]:
class NaverReviewDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          padding='max_length',
          return_attention_mask=True,
          return_tensors='pt',
          truncation=True,
        )

        return {
          'text': text,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'labels': torch.tensor(label, dtype=torch.long)
        }
    
    def __len__(self):
        return len(self.texts)


In [14]:
dataset = NaverReviewDataset(naver_data['document'], naver_data['label'], tokenizer, 100)
train_set, valid_set, test_set = torch.utils.data.random_split(dataset, [40000, 5000, 5000])

In [15]:
train_set[0]

{'attention_mask': tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]),
 'input_ids': tensor([9086, 8299,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
            3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
            3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
            3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
            3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
            3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
            3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
            3,    3,    3,    3,    3,    3,    3,    3

In [16]:
batch_size = 8

train_dataloader = DataLoader(train_set, batch_size=batch_size,
                        shuffle=True)

valid_dataloader = DataLoader(valid_set, batch_size=batch_size,
                        shuffle=True)

test_dataloader = DataLoader(test_set, batch_size=batch_size,
                        shuffle=True)

In [22]:
sample_data = next(iter(test_dataloader))
sample_data

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 

## 5-2. Model Settings

 - Model의 환경을 설정합니다.

In [18]:
gpt2lm_model.train()

learning_rate = 1e-5
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gpt2lm_model.parameters(), lr=learning_rate)

device = 'cuda'

epochs = 10
count = 0

In [23]:
sample_inputs = sample_data['input_ids']
sample_outputs = gpt2lm_model(sample_inputs, labels=sample_inputs)

In [24]:
sample_outputs['loss']

tensor(9.5358, grad_fn=<NllLossBackward0>)

## 5-3. Model Training

 - Model의 학습을 시작합니다.

In [ ]:
tot_train_loss = 0.0
tot_valid_loss = 0.0
prev_valid_loss = 10000

print('KoGPT-2 Training Start!')

for epoch in range(epochs):
    for batch, train_data in enumerate(train_dataloader):
        # train data를 모델에 입력하여 출력 값을 얻습니다.
        gpt2lm_model.to(device)
        train_inputs = train_data['input_ids'].to(device)
        train_outputs = gpt2lm_model(train_inputs, labels=train_inputs) # train_outputs = (train_loss, train_logits, train_past_hidden_states)
        # label과 data를 똑같이 넣어주면 한칸씩 밀림
        
        train_loss, _ = train_outputs[:2]
        
        # valid data를 모델에 입력하여 출력 값을 얻습니다.
            
        valid_data = next(iter(valid_dataloader))

        gpt2lm_model.to(device)
        valid_inputs = valid_data['input_ids'].to(device)     
        valid_outputs = gpt2lm_model(valid_inputs, labels=valid_inputs)
        
        valid_loss, _ = valid_outputs[:2]
        
        gpt2lm_model.to(device)
        
        # train loss를 토대로 모델을 학습합니다.
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        tot_train_loss += train_loss.item()
        tot_valid_loss += valid_loss.item()
               
        # 200 batch 마다 학습 상황을 화면에 출력합니다.
        if count % 200 == 0:
            cnt = ((count+1) * batch_size)
            current_train_loss = tot_train_loss / cnt
            current_valid_loss = tot_valid_loss / cnt
            
            print(f'epoch : %5d | batch : %5d | train_loss : %.5f | valid_loss : %.5f' %(epoch+1, batch+1, current_train_loss, current_valid_loss))
            
            tot_train_loss = 0.0
            tot_valid_loss = 0.0
            
            count = 0
            
            # 이전 valid_loss 보다 현재의 valid_loss가 더 낮을 경우, 모델을 저장합니다.
            if prev_valid_loss > current_valid_loss:
                prev_valid_loss = current_valid_loss
                torch.save(gpt2lm_model.state_dict(), f'./KoGPT-model.pth')
        
        count += 1

KoGPT-2 Training Start!
epoch :     1 | batch :     1 | train_loss : 1.19892 | valid_loss : 1.19924
epoch :     1 | batch :   201 | train_loss : 0.13436 | valid_loss : 0.13313
epoch :     1 | batch :   401 | train_loss : 0.11667 | valid_loss : 0.11756
epoch :     1 | batch :   601 | train_loss : 0.11221 | valid_loss : 0.11503
epoch :     1 | batch :   801 | train_loss : 0.11107 | valid_loss : 0.11307
epoch :     1 | batch :  1001 | train_loss : 0.10849 | valid_loss : 0.10750
epoch :     1 | batch :  1201 | train_loss : 0.11019 | valid_loss : 0.11094
epoch :     1 | batch :  1401 | train_loss : 0.10775 | valid_loss : 0.11051
epoch :     1 | batch :  1601 | train_loss : 0.10634 | valid_loss : 0.11045
epoch :     1 | batch :  1801 | train_loss : 0.10588 | valid_loss : 0.10863
epoch :     1 | batch :  2001 | train_loss : 0.10417 | valid_loss : 0.10775
epoch :     1 | batch :  2201 | train_loss : 0.10940 | valid_loss : 0.10763
epoch :     1 | batch :  2401 | train_loss : 0.10642 | valid_los

In [ ]:
train_set[5]

In [ ]:
kogpt_load_path = f"./KoGPT-model.pth"

gpt2lm_model.load_state_dict(torch.load(kogpt_load_path))

In [ ]:
gpt2lm_model.to(device)

sample_text = "정말 재미"

tokens = tokenizer.tokenize(sample_text)
token_ids = tokenizer.encode(sample_text)

gen_ids = gpt2lm_model.generate(torch.tensor([token_ids]).to(device),
                           max_length=127,
                           repetition_penalty=1.0,
                           num_beams=5)

generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

In [ ]:
import re

p = re.compile('<pad>')
re.sub(p, '', generated)

# 6. Fine Tuning 2 (Classification Task)

 - Dateset을 가져옵니다.

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>', padding_side='left') 

In [ ]:
batch_size = 16

naver_data = get_naver_review_examples()

dataset = NaverReviewDataset(naver_data['document'], naver_data['label'], tokenizer, 100)
train_set, valid_set, test_set = torch.utils.data.random_split(dataset, [40000, 5000, 5000])

train_dataloader = DataLoader(train_set, batch_size=batch_size,
                        shuffle=True)

valid_dataloader = DataLoader(valid_set, batch_size=batch_size,
                        shuffle=True)

test_dataloader = DataLoader(test_set, batch_size=batch_size,
                        shuffle=True)

 - GPT Classifier를 정의합니다.

In [ ]:
class GPT2SentimentClassifier(torch.nn.Module):

    def __init__(self, n_classes):
        super(GPT2SentimentClassifier, self).__init__()

        self.gpt_model = GPT2Model.from_pretrained('skt/kogpt2-base-v2')
        self.drop = torch.nn.Dropout(p=0.1)
        self.out = torch.nn.Linear(self.gpt_model.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        hidden_states = self.gpt_model(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
        last_hidden_state = hidden_states[0]
        
        output = self.drop(last_hidden_state[:, -1, :])

        return self.out(output)


 - Model의 환경을 설정합니다.

In [ ]:
gpt_clf = GPT2SentimentClassifier(n_classes=1)
gpt_clf.train()

learning_rate = 5e-5
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(gpt_clf.parameters(), lr=learning_rate)

device = 'cuda'

epochs = 1
count = 0

 - 정확도 계산 함수를 정의합니다. 

In [ ]:
def cal_correct_num(predicts, labels):
    predicts_ = predicts >= 0.5
    correct_num = torch.sum(predicts_ == labels)
        
    return correct_num

 - Model의 학습을 시작합니다

In [ ]:
tot_train_loss = 0.0
tot_valid_loss = 0.0

train_correct_num = 0
valid_correct_num = 0

prev_valid_loss = 10000

print('KoGPT-2 Training Start!')

for epoch in range(epochs):
    for batch, train_data in enumerate(train_dataloader):
        # train data를 모델에 입력하여 출력 값을 얻습니다.
        gpt_clf.to(device)
        train_inputs = train_data['input_ids'].to(device)
        train_masks = train_data['attention_mask'].to(device)
        train_labels = train_data['labels'].to(device)
        
        train_outputs = gpt_clf(train_inputs, train_masks)
        
        train_loss = criterion(train_outputs.view(-1), train_labels.float())
        
        # valid data를 모델에 입력하여 출력 값을 얻습니다.
            
        valid_data = next(iter(valid_dataloader))

        gpt_clf.to(device)
        valid_inputs = valid_data['input_ids'].to(device)    
        valid_masks = valid_data['attention_mask'].to(device)
        valid_labels = valid_data['labels'].to(device)
        
        valid_outputs = gpt_clf(valid_inputs, valid_masks)
        
        valid_loss = criterion(valid_outputs.view(-1), valid_labels.float())
        
        gpt_clf.to(device)
        
        # train loss를 토대로 모델을 학습합니다.
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        tot_train_loss += train_loss.item()
        tot_valid_loss += valid_loss.item()
        
        train_correct_num += cal_correct_num(torch.sigmoid(train_outputs.view(-1)), train_labels.float())
        valid_correct_num += cal_correct_num(torch.sigmoid(valid_outputs.view(-1)), valid_labels.float())
               
        # 200 batch 마다 학습 상황을 화면에 출력합니다.
        if count % 200 == 0:
            cnt = ((count+1) * batch_size)
            current_train_loss = tot_train_loss / cnt
            current_valid_loss = tot_valid_loss / cnt
            
            train_acc = train_correct_num / cnt
            valid_acc = valid_correct_num / cnt
            
            print(f'epoch : %5d | batch : %5d | train_loss : %.5f | valid_loss : %.5f | train_acc : %.5f | valid_acc : %.5f' %(epoch+1, batch+1, current_train_loss, current_valid_loss, train_acc, valid_acc))
            
            tot_train_loss = 0.0
            tot_valid_loss = 0.0
            
            train_correct_num = 0
            valid_correct_num = 0
            
            count = 0
            
            # 이전 test_loss 보다 현재의 test_loss가 더 낮을 경우, 모델을 저장합니다.
            if prev_valid_loss > current_valid_loss:
                prev_valid_loss = current_valid_loss
                torch.save(gpt_clf.state_dict(), f'./KoGPT-Classifier-model.pth')
        
        count += 1